In [25]:
import os
from PIL import Image
import numpy as np

openslide_path = r'C:\Users\Donald\Documents\openslide-win64-20230414\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

You need to do get crop data before running this code. I use a matlab function I made with the 2.5x or 10x tiff images.

Function is here: \\10.99.68.178\andreex\students\Donald Monkey fetus\codes\monkey\crop1024_monkey.m

In [26]:
pth = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\1024tiles\new_tiles_11_3'  # path of ndpi files

pth_mat = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\1024tiles\new_tiles_11_3\10x\matfiles'  # path .mat files with crop data (got from matlab file)

crop_mats = [_ for _ in os.listdir(pth_mat) if _.endswith('mat')]
WSIs = [_ for _ in os.listdir(pth) if _.endswith('ndpi')]

In [27]:
crop_mats

['crop_monkey_fetus_40_0371_Lung.mat',
 'crop_monkey_fetus_40_0431_GI.mat',
 'crop_monkey_fetus_40_0451_kidney.mat',
 'crop_monkey_fetus_40_0507_Thyroid.mat',
 'crop_monkey_fetus_40_0519_Ganglion.mat',
 'crop_monkey_fetus_40_0532_heart.mat',
 'crop_monkey_fetus_40_0612_brain.mat',
 'crop_monkey_fetus_40_0640_bone.mat',
 'crop_monkey_fetus_40_0642_bone2.mat']

In [28]:
WSIs

['monkey_fetus_40_0371_Lung.ndpi',
 'monkey_fetus_40_0431_GI.ndpi',
 'monkey_fetus_40_0451_kidney.ndpi',
 'monkey_fetus_40_0507_Thyroid.ndpi',
 'monkey_fetus_40_0519_Ganglion.ndpi',
 'monkey_fetus_40_0532_heart.ndpi',
 'monkey_fetus_40_0612_brain.ndpi',
 'monkey_fetus_40_0640_bone.ndpi',
 'monkey_fetus_40_0642_bone2.ndpi']

In [29]:
res20x=0.4416
res10x=1

ratio = res10x/res20x

outpth = os.path.join(pth,'tiles256')
if not os.path.exists(outpth):
    os.mkdir(outpth)

In [30]:
from scipy.io import loadmat

file_format = '.tif'

for i, file in enumerate(crop_mats):
    mat_file_name = os.path.join(pth_mat, file)

    WSI_path = os.path.join(pth, WSIs[i])

    f_out_name = f'{os.path.join(os.path.join(outpth,WSIs[i][:-5]))}{file_format}'

    if os.path.exists(f_out_name):
        print(f'Skipping cropping for {WSIs[i]} ({i+1}/{len(crop_mats)})... Already exists')

    else:
        print(f'Cropping {WSIs[i]} ({i+1}/{len(crop_mats)})')

        # print(mat_file_name)
        # print(WSI_path)

        WSI_slide = openslide.OpenSlide(WSI_path)

        wsi_width, wsi_height = WSI_slide.level_dimensions[0]

        data = loadmat(mat_file_name)

        # crop_width = int(data['crop_width']*ratio)
        # crop_height = int(data['crop_height']*ratio)


        crop_width = 256
        crop_height = 256
        crop_x = int(data['crop_x']*ratio)
        crop_y = int(data['crop_y']*ratio)

        lower_x = crop_x
        upper_y = crop_y
        upper_x = crop_x + crop_width
        lower_y = crop_y - crop_height

        # print(crop_x)
        # print(crop_y)
        # print(crop_width)
        # print(crop_height)

        region = WSI_slide.read_region((crop_x, crop_y), 0, (crop_width, crop_height)) # level = 0 in middle
        region_image = Image.new('RGB', region.size)
        region_image.paste(region, (0, 0))

        region_image.save(f_out_name)
        WSI_slide.close()


Cropping monkey_fetus_40_0371_Lung.ndpi (1/9)
Cropping monkey_fetus_40_0431_GI.ndpi (2/9)
Cropping monkey_fetus_40_0451_kidney.ndpi (3/9)
Cropping monkey_fetus_40_0507_Thyroid.ndpi (4/9)
Cropping monkey_fetus_40_0519_Ganglion.ndpi (5/9)
Cropping monkey_fetus_40_0532_heart.ndpi (6/9)
Cropping monkey_fetus_40_0612_brain.ndpi (7/9)
Cropping monkey_fetus_40_0640_bone.ndpi (8/9)
Cropping monkey_fetus_40_0642_bone2.ndpi (9/9)
